In [ ]:
import os 
import sys 
sys.path.append("../")
import re 
import json 
import numpy as np 
import pandas as pd 
from collections import defaultdict, Counter
import torch 
import datasets
import nltk  # Here to have a nice missing dependency error message early on
import datasets
from datasets import load_dataset

import evaluate
import transformers
from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BertTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, is_offline_mode, send_example_telemetry
from transformers.utils.versions import require_version

from tqdm import tqdm
from sacrebleu.metrics import BLEU, CHRF, TER
from bert_score import BERTScorer
from torch.utils.data import Dataset, DataLoader

from utils import recover_sentence

In [ ]:
torch.manual_seed(0)

In [ ]:
test_file = "../datasets/esconv/test.csv"
data_files = {}
data_files["test"] = test_file
extension = test_file.split(".")[-1]
raw_datasets = load_dataset(
    extension,
    data_files=data_files,
    use_auth_token=None,
)

In [ ]:
gt_df = pd.read_csv("../datasets/esconv/test.csv")
gt_texts = gt_df["target"].values
strategy_labels = gt_df["strategy"].values

In [ ]:
model_path = "../train_esconv_outputs/bart_condition1"
config = AutoConfig.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda:0")

In [ ]:
input_column = "history"
target_column = "target"
input_truncation_side = "left"
max_source_length = 384
max_target_length = 128
padding = "max_length"
ignore_pad_token_for_loss = True
column_names = raw_datasets["test"].column_names
control_columns = ['strategy']
fp16 = True
class_control_mapping = json.load(open("../aux_files/control_token_mapping_esconv.json", encoding="utf-8"))
used_control_mappings = {}
extra_control_tokens = set(["[speaker1]", "[speaker2]"])
for col in control_columns:
    assert col in class_control_mapping
    # sth like {0: "", 1: "[positive]", 2: "[negative]"}
    for k in class_control_mapping[col]:
        if k != "" and k.count("[") == 1:  # do not add composed tokens like "[-positive][-negative]"
            extra_control_tokens.add(k)
    mapping0 = {v: k for k, v in class_control_mapping[col].items()}
    used_control_mappings[col] = mapping0


In [ ]:
def preprocess_function(examples):
    # remove pairs where at least one record is None

    inputs, targets = [], []
    for i in range(len(examples[input_column])):
        if examples[input_column][i] and examples[target_column][i]:
            inputs.append(examples[input_column][i])
            targets.append(examples[target_column][i])
            for control_column in control_columns:
                control_label = examples[control_column][i]
                control_token = used_control_mappings[control_column][control_label]
                inputs[-1] = inputs[-1] + control_token
            
    tokenizer.truncation_side = input_truncation_side
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    # labels = tokenizer(text_target=targets, max_length=max_target_length, padding=padding, truncation=True)
    tokenizer.truncation_side = 'right'
    labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length" and ignore_pad_token_for_loss:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
predict_dataset = raw_datasets["test"]
predict_dataset = predict_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on prediction dataset",
)

In [ ]:
# Data collator
label_pad_token_id = -100 if ignore_pad_token_for_loss else tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 if fp16 else None,
)

In [ ]:
data_loader = DataLoader(predict_dataset, batch_size=32, num_workers=4, collate_fn=data_collator, pin_memory=True)

In [ ]:
out_dir = os.path.join(model_path, "infer_sampling/")
os.makedirs(out_dir, exist_ok=True)

In [ ]:
all_outputs = []
for batch_id, batch in enumerate(tqdm(data_loader)):
    input_ids = batch["input_ids"].to(model.device)
    outputs = model.generate(input_ids, do_sample=True, max_length=128, top_p=0.5, num_return_sequences=1)
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    all_outputs.extend(outputs)
with open(out_dir+"topp_0.5.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(all_outputs))

In [ ]:
with open(out_dir+"topp_0.5.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(all_outputs))
